In [1]:
import random
import math
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')


In [2]:
class Neuron():

    def __init__(self,position_in_layer,is_output_neuron=False):
        self.weights = []
        self.inputs = []
        self.output = None
        self.updated_weights = []
        self.is_output_neuron = is_output_neuron
        self.delt = None
        self.position_in_layer = position_in_layer
    
    def attach_to_output(self,neurons):
        self.output_neurons = []
        for n in neurons:
            self.output_neurons.append(n)
    
    def sigmoid(self,x):
        return 1 / (1+math.exp(-x))
    
    def init_weights(self,num_input):
        for _ in range(num_input):
            self.weights.append(random.uniform(0,1))

    def predict(self,row):
        self.inputs = []
        activation = 0
        for weight, feature in zip(self.weights,row):
            self.inputs.append(feature)
            activation = activation + weight * feature
        self.output = self.sigmoid(activation)
        return self.output
    
    def update_neuron(self):
        self.weights = []
        for new_weights in self.updated_weights:
            self.weights.append(new_weights)

    def calculate_update(self,learning_rate,target):
        if self.is_output_neuron:
            self.delta = (self.output - target) * self.output*(1-self.output)
        else:
            delta_sum = 0
            cur_weights_index = self.position_in_layer
            for output_neuron in self.output_neurons:
                delta_sum = delta_sum + (output_neuron.delta * output_neuron.weights[cur_weights_index])
            self.delta = delta_sum * self.output*(1-self.output)

        self.updated_weights = []

        for cur_weights,cur_input in zip(self.weights,self.inputs):
            gradient = self.delta * cur_input
            new_weight = cur_weights - learning_rate * gradient
            self.updated_weights.append(new_weight)
        

In [3]:
class Layer():
    
    def __init__(self,num_neuron,is_output_layer = False):
        self.is_output_layer = is_output_layer
        self.neurons = []
        for i in range(num_neuron):
            neuron = Neuron(i,is_output_neuron=is_output_layer)
            self.neurons.append(neuron)
    
    def attach(self,layer):
        for in_neuron in self.neurons:
            in_neuron.attach_to_output(layer.neurons)
    
    def init_layer(self,num_input):
        for neuron in self.neurons:
            neuron.init_weights(num_input)
    
    def predict(self,row):
        activations = [neuron.predict(row) for neuron in self.neurons]
        return activations

In [4]:
class MultiLayerPerceptron():
    
    def __init__(self,learning_rate = 0.01,num_iterations = 100):
        self.layers = []
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.errors = []
        self.loss = []
    
    def eta(self,x):
        return np.maximum(x, 0.0000000001)

    def entropy_loss(self, y, y_pred):
        y_pred_inv = self.eta(1 - y_pred)
        y_inv = self.eta(1 - y)
        loss = -np.average(y*np.log(y_pred) +
                           y_inv * np.log(y_pred_inv))
        return loss

    def add_output_layer(self,num_neuron):
        self.layers.insert(0,Layer(num_neuron,is_output_layer=True))
    
    def add_hidden_layer(self,num_neuron):
        hidden_layer = Layer(num_neuron)
        hidden_layer.attach(self.layers[0])
        self.layers.insert(0,hidden_layer)

    def update_layers(self,target):
        for layer in reversed(self.layers):
            for neuron in layer.neurons:
                neuron.calculate_update(self.learning_rate,target)
        for layer in self.layers:
            for neuron in layer.neurons:
                neuron.update_neuron()
        
    def fit(self,X,y):
        num_row,num_feature = X.shape
        X = X.values.tolist()
        self.layers[0].init_layer(num_feature)

        for i in range(1,len(self.layers)):
            num_input = len(self.layers[i-1].neurons)
            self.layers[i].init_layer(num_input)
        
        for _ in range(self.num_iterations):
            r_i = random.randint(0,num_row-1)
            row = X[r_i]
            y_pred = self.predict(row)
            target = y[r_i]
            self.update_layers(target)

            if _ % 1000 == 0:
                total_error = 0
                for r_i in range(num_row):
                    row = X[r_i]
                    y_pred = self.predict(row)
                    self.loss.append(self.entropy_loss(y[r_i][0],y_pred))
                    error = (y[r_i][0]-y_pred)
                    total_error = total_error + error ** 2
                mean_error = total_error/num_row
                print(f"Iteration {_} with mean error = {mean_error}")
    
    def predict(self,row):
        activations = self.layers[0].predict(row)

        for _ in range(1,len(self.layers)):
            activations = self.layers[_].predict(activations)
        
        outputs = []

        for activation in activations:
            if activation >= 0.5:
                outputs.append(1)
            else:
                outputs.append(0)
        return outputs[0]          


In [5]:
scaler = preprocessing.StandardScaler()

In [6]:
df = pd.read_csv("heart_disease.csv")
X_df = df.drop(["condition"], axis=1)
X_pre = scaler.fit_transform(X_df)
X = pd.DataFrame(X_pre, columns=df.columns[:-1])
y = df["condition"].values.reshape(X.shape[0], 1)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2)

In [8]:
clf = MultiLayerPerceptron(learning_rate=0.1, num_iterations=10000)
clf.add_output_layer(num_neuron=1)
clf.add_hidden_layer(num_neuron=3)
clf.fit(X_train, y_train)


Iteration 0 with mean error = 0.5569620253164557
Iteration 1000 with mean error = 0.38396624472573837
Iteration 2000 with mean error = 0.20253164556962025
Iteration 3000 with mean error = 0.15611814345991562
Iteration 4000 with mean error = 0.1350210970464135
Iteration 5000 with mean error = 0.12658227848101267
Iteration 6000 with mean error = 0.1350210970464135
Iteration 7000 with mean error = 0.1350210970464135
Iteration 8000 with mean error = 0.12236286919831224
Iteration 9000 with mean error = 0.11814345991561181


In [9]:
acc = 0
for i in range(X_test.shape[0]):
     row = np.asarray(X_test)[i]
     if clf.predict(row) == y_test[i]:
          acc += 1
print(acc/X_test.shape[0])

0.8333333333333334
